In [20]:
data_dir='/home/ubuntu/intelliscope/ecg_data/'# fill # data_dir/ecg_other_train, ecg_other_val, ... ecg_other_train_imb, ..should be there
model_dir='/home/ubuntu/intelliscope/notebooks/physionet2017/ecg/models/'

In [21]:
PARAMS = {
          'epochs' : 15,
         'validation_split':0.15,
         'batch_size' : 32,
            'model': 'simple',
          'data_balanced':False,
            'num_epochs':15,
            'opt':'adam',
            'tags': ['instance_norm added', 'dropout added'],
         }

In [22]:
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization

from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import tensorflow as tf

In [23]:


#k1,k2=100,100
k1,k2=272, 462

model_simple1=Sequential([
    #tf.keras.layers.experimental.preprocessing.Resizing(k1, k2, interpolation="bilinear")
    Lambda(lambda image: tf.image.resize( image, (64, 64), method = tf.image.ResizeMethod.BICUBIC, preserve_aspect_ratio = True),input_shape=(k1, k2,3)),
    Conv2D(16, (5, 5),activation='relu'),
    Conv2D(32, (5, 5),activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3),activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(3840, activation='relu'),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(2,activation='softmax')])

model_simple2=Sequential([
    #tf.keras.layers.experimental.preprocessing.Resizing(k1, k2, interpolation="bilinear")
    Lambda(lambda image: tf.image.resize( image, (64, 64), method = tf.image.ResizeMethod.BICUBIC, preserve_aspect_ratio = True),input_shape=(k1, k2,3)),
    Conv2D(16, (5, 5),activation='relu'),
    InstanceNormalization(axis=3, center=True, scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform"),
    Dropout(0.2),
    Conv2D(32, (5, 5),activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    InstanceNormalization(axis=3, center=True, scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform"),
    Dropout(0.2),
    Conv2D(64, (3, 3),activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    InstanceNormalization(axis=3, center=True, scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform"),
    Dropout(0.2),
    Flatten(),
    Dense(3840, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64,activation='relu'),
    Dropout(0.5),
    Dense(2,activation='softmax')])


model=model_simple2
model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=PARAMS['opt'],
              metrics=[keras.metrics.CategoricalAccuracy(), keras.metrics.Recall(),keras.metrics.Precision()])

PARAMS['model']=model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_7 (Lambda)            (None, 38, 64, 3)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 34, 60, 16)        1216      
_________________________________________________________________
instance_normalization_9 (In (None, 34, 60, 16)        32        
_________________________________________________________________
dropout_19 (Dropout)         (None, 34, 60, 16)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 30, 56, 32)        12832     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 15, 28, 32)        0         
_________________________________________________________________
instance_normalization_10 (I (None, 15, 28, 32)       

In [24]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

batch_size=PARAMS['batch_size']

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        data_dir+'ecg_other_train_imb/',  # this is the target directory
        target_size=(k1, k2),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

validation_generator = val_datagen.flow_from_directory(
        data_dir+'ecg_other_val_imb/',
        target_size=(k1, k2),
        batch_size=batch_size,
        class_mode='categorical')

history=model.fit_generator(
        train_generator,
        #steps_per_epoch=2000 // batch_size,
        epochs=PARAMS['num_epochs'],
        validation_data=validation_generator,
        #validation_steps=800 // batch_size
)



Found 4078 images belonging to 2 classes.
Found 873 images belonging to 2 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 128 steps, validate for 28 steps
Epoch 1/15
128/128 [==============================] - 186s 1s/step - loss: 0.4887 - categorical_accuracy: 0.8396 - recall_3: 0.8396 - precision_3: 0.8396 - val_loss: 0.5122 - val_categorical_accuracy: 0.8545 - val_recall_3: 0.8545 - val_precision_3: 0.8545
Epoch 2/15
128/128 [==============================] - 182s 1s/step - loss: 0.4008 - categorical_accuracy: 0.8762 - recall_3: 0.8762 - precision_3: 0.8762 - val_loss: 0.4155 - val_categorical_accuracy: 0.8545 - val_recall_3: 0.8545 - val_precision_3: 0.8545
Epoch 3/15
128/128 [==============================] - 161s 1s/step - loss: 0.3840 - categorical_accuracy: 0.8764 - recall_3: 0.8764 - precision_3: 0.8764 - val_loss: 0.4179 - val_categorical_accuracy: 0.8545 - val_recall_3: 0.8545 - val_precision_3: 0.8545
Epoch 4/15
128/128 [==============================] -

In [25]:

#!pip install neptune-client
import os
import neptune
NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNjBlODllY2ItMDcyZC00OGNmLThjZWYtZjM2MjE4Y2M5ZDM0In0="
os.environ['NEPTUNE_PROJECT']="icassp/ECG"
#os.environ['NEPTUNE_NOTEBOOK_ID']="4897a223-8a9f-4f00-8cec-22eda690858d"
os.environ['NEPTUNE_NOTEBOOK_PATH']='ECG-simple (imbalanced).ipynb'

neptune.init('icassp/ECG',NEPTUNE_API_TOKEN)
exp = neptune.create_experiment(description='ECG-simple (imbalanced)',params=PARAMS,upload_source_files=['ECG-simple (imbalanced).ipynb'],tags=PARAMS['tags'],upload_stdout=True)

model.save(f'{model_dir}{str(exp)}.h5')
neptune.log_artifact(f'{model_dir}{str(exp)}.h5')

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/icassp/ECG/e/ECG-27


In [26]:
no=3
for i in range(len(history.history['loss'])):
    neptune.log_metric('loss',history.history['loss'][i])
    neptune.log_metric('val loss',history.history['val_loss'][i])
    neptune.log_metric('acc',history.history['categorical_accuracy'][i])
    neptune.log_metric('val_acc',history.history['val_categorical_accuracy'][i])
    neptune.log_metric('precision',history.history['precision_%d'%no][i])
    neptune.log_metric('val_precision',history.history['val_precision_%d'%no][i])
    neptune.log_metric('recall',history.history['recall_%d'%no][i])
    neptune.log_metric('val_recall',history.history['val_recall_%d'%no][i])

In [ ]:
for i in range(len(history.history['loss'])):
    neptune.log_metric('loss',history.history['loss'][i])
    neptune.log_metric('val loss',history.history['val_loss'][i])
    neptune.log_metric('acc',history.history['categorical_accuracy'][i])
    neptune.log_metric('val_acc',history.history['val_categorical_accuracy'][i])
    neptune.log_metric('precision',history.history['precision'][i])
    neptune.log_metric('val_precision',history.history['val_precision'][i])
    neptune.log_metric('recall',history.history['recall'][i])
    neptune.log_metric('val_recall',history.history['val_recall'][i])

In [27]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        data_dir+'ecg_other_test_imb/',  # this is the target directory
        target_size=(k1, k2),  # all images will be resized to 150x150
        batch_size=len(os.listdir(data_dir+'ecg_other_test_imb/1'))+len(os.listdir(data_dir+'ecg_other_test_imb/0')),
        class_mode='categorical')

X,y = test_generator.next()
prediction = model.predict(X)

import numpy as np

y_test=np.argmax(y,axis=1)
y_pred=prediction
from neptunecontrib.monitoring.metrics import *

threshold = 0.5

log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
log_classification_report(y_test, y_pred[:, 1] > threshold)
log_class_metrics(y_test, y_pred[:, 1] > threshold)
log_class_metrics_by_threshold(y_test, y_pred[:, 1])
log_roc_auc(y_test, y_pred)
log_precision_recall_auc(y_test, y_pred)
log_brier_loss(y_test, y_pred[:, 1])
log_log_loss(y_test, y_pred)
log_ks_statistic(y_test, y_pred)
log_cumulative_gain(y_test, y_pred)
log_lift_curve(y_test, y_pred)
log_prediction_distribution(y_test, y_pred[:, 1])


Found 874 images belonging to 2 classes.


Invalid metric value: nan for channel positive_predictive_value. Metrics with nan or +/-inf values will not be sent to server
Invalid metric value: nan for channel false_discovery_rate. Metrics with nan or +/-inf values will not be sent to server


In [28]:
neptune.stop()